In [ ]:
import os

if os.getcwd() == '/Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision/notebooks':
    os.chdir('/Users/matildamwendwa/Desktop/Desktop - Admin’s MacBook Pro/Python_Projects/microvision')
    print("Changed!!")

print("Current working directory:", os.getcwd())

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
%pip install chromadb sentence-transformers tqdm pandas numpy matplotlib networkx  faiss --quiet

#### SETTING UP THE ENVIRONMENT


In [ ]:
import os
import glob
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from torch import embedding
from tqdm import tqdm

# ------ CONFIGURATIONS -----
config = {
        "DATA_DIR": "data",
        "DATASET_NAME": "OpenStack",
        "DATASET_LOG": "_full.log",
        "ENRICHED_CSV": "_enriched.csv",
        "TEMPLATES_CSV": "_templates.csv",
        "TEMPLATES_JSON": "_templates.json",
        "MAX_LINES": None,  # Set to None to process all lines
        "PERSISTENCE_PATH": "persistence",
        "DRAIN_PATH": "drain3_state",
                
        "EMBEDDINGS_CSV": "_embeddings.csv",
        "CHROMA_PERSIST_DIR": "chroma_storage",
        "COLLECTION_NAME": "_collection"
}

os.makedirs(config["DATA_DIR"], exist_ok=True)

# ------ LOADING EMBEDDINGS DATAFRAME -----
embeddings_csv_path = f"{config['DATA_DIR']}/{config['DATASET_NAME']}{config['DATASET_LOG']}{config['EMBEDDINGS_CSV']}"
embeddings_df = pd.read_csv(embeddings_csv_path)
embeddings_df["embedding"] = embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=","))
print(f"✅ Loaded embeddings DataFrame with {len(embeddings_df)} rows from {embeddings_csv_path}")

# ------ CONNECTING TO CHROMADB -----

chroma_persist_dir = f"{config['DATA_DIR']}/{config['PERSISTENCE_PATH']}/{config['CHROMA_PERSIST_DIR']}"
client = chromadb.PersistentClient(path=chroma_persist_dir)
print(f"✅ Connected to ChromaDB client, {chroma_persist_dir}")

collection_name = f"{config['DATASET_NAME']}{config['DATASET_LOG']}{config['COLLECTION_NAME']}"
# print(f"Chroma contains collections: {client.list_collections()}")

collection = client.get_collection(collection_name)
print(f"📦 Active collection: {collection.name}, embeddings: {collection.count()}")

In [ ]:
# ------ LOADING THE SENTENCE TRANSFORMER MODEL -----
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
print(f"✅ Model Loaded: {model_name}")

#### UTILITY FUNCTIONS USED IN THE RAG ENGINE

In [ ]:
# ------- QUICK VERIFICATION OF EMEDDINGS

def ensure_unique_ids(df, id_col):
    """
    Ensure unique identifiers for all templates (string IDs).
    Converts duplicates into distinct suffixed IDs.
    """
    tqdm.write("🔢 Ensuring unique template IDs...")
    seen = {}
    new_ids = []
    for val in df[id_col].astype(str):
        count = seen.get(val, 0)
        if count > 0:
            new_ids.append(f"{val}_{count}")
        else:
            new_ids.append(val)
        seen[val] = count + 1
    df[id_col] = new_ids
    return df

def validate_embeddings_shape(df, col):
    """
    Validate and convert stored embeddings into numpy arrays.
    """
    tqdm.write("🧩 Validating embedding structures...")
    return np.vstack(df[col].apply(lambda e: np.array(e, dtype=float)))

In [ ]:
# Validating the embeddings dataframe
embeddings_df = ensure_unique_ids(embeddings_df, id_col="template_id")
embeddings_array = validate_embeddings_shape(embeddings_df, col="embedding")
print(f"✅ Embeddings shape: {embeddings_array.shape}") 



#### Retrieval Module Functions

Using ChromaDB's Internal ANN retriever

In [ ]:
def semantic_retrieve_topk(collection, query_embeddings, top_k, metadata_filter, include_distances):
    # tqdm.write(f"🔍 Retrieving top-{top_k} neighbors from Chroma (filters={metadata_filter})")

    query_embeddings = np.atleast_2d(query_embeddings).tolist()
    
    results = collection.query(
        query_embeddings=query_embeddings,
        n_results=top_k,
        where=metadata_filter if isinstance(metadata_filter, dict) else None
    )

    output = []
    for i, _ in enumerate(query_embeddings):
        batch = {
            "ids": results["ids"][i],
            "documents": results["documents"][i],
            "metadatas": results["metadatas"][i],
        }
        if include_distances and "distances" in results:
            batch["distances"] = results["distances"][i]
        output.append(batch)
    
    # print("Retrieved Output: ", output)
    return output


#### Post-Retrieval Functions

1. Metadata Correlation Scoring


2. Temporal Validation Function: it validates the causal direction between two services based on timestamps. 

if: ts_a precedes ts_b within window the it returns "A->B"
elif: ts_b precedes ts_a within window the it returns "B->A"
else: none

3. Semantic and Metadata Similarity Score Functions

3.1 Score Normalization Function: Normalizes list/array of scores into [0, 1]

3.2 Fusion Function: Fuses semantic and metadata similarity scores into a hybrid score, also has a parameter alpha
that represents the weight for semantic similarity (0–1)

In [ ]:
# -------- METADATA SCORING FUNCTIONS --------

import ast

def normalize_metadata(meta):
    """Normalize metadata dict for case-insensitive matching and parsing."""
    if isinstance(meta, str):
        try:
            meta = ast.literal_eval(meta)
        except Exception:
            return {}
    if not isinstance(meta, dict):
        # Handles Pandas Series (row) or other structures
        if hasattr(meta, "to_dict"):
            meta = meta.to_dict()
        else:
            return {}

    # Convert all keys to lowercase and string values
    return {str(k).lower(): str(v).strip() for k, v in meta.items()}

def compute_metadata_weights(df, fields):
    weights = {f: 1 / max(df[f].nunique(), 1) for f in fields}
    total = sum(weights.values())
    weights = {k: v / total for k, v in weights.items()}
    tqdm.write(f"⚖️ Metadata weights: {weights}")
    return weights


def metadata_correlation_score(src_meta, target_meta, fields, weights):
    src = {k.lower(): str(v).strip() for k, v in src_meta.items()}
    target = {k.lower(): str(v).strip() for k, v in target_meta.items()}

    score, total = 0.0, 0.0
    for f in fields:
        f_lower = f.lower()
        total += weights.get(f, 0)
        if src.get(f_lower) and src.get(f_lower) == target.get(f_lower):
            score += weights.get(f, 0)
    return score / total if total > 0 else 0.0

In [ ]:
def safe_timestamp(row):
    date, time = row.get("date"), row.get("time")
    if pd.notna(date) and pd.notna(time):
        try:
            return pd.to_datetime(f"{date} {time}", errors="coerce")
        except Exception:
            return pd.NaT
    return pd.NaT

# -------- TEMPORAL DIRECTION VALIDATION FUNCTION --------

from datetime import datetime
import pandas as pd
def parse_ts(ts):
    if isinstance(ts, (datetime, pd.Timestamp)):
        return ts
    try:
        return pd.to_datetime(str(ts), errors="coerce")
    except Exception:
        return pd.NaT
        
def temporal_direction(ts_a, ts_b, max_window):
    # Debug info
    # print(f"ts_a: {ts_a}, ts_b: {ts_b}")

    # Handle missing or invalid timestamps
    # if pd.isna(ts_a) or pd.isna(ts_b):
    #     return "none", None

    # Safe parsing function


    ts_a = parse_ts(ts_a)
    ts_b = parse_ts(ts_b)

    # Ensure both are valid
    # if pd.isna(ts_a) or pd.isna(ts_b):
    #     return "none", None

    # Compute delta in seconds
    delta = (ts_b - ts_a).total_seconds()


    if 0 < delta <= max_window:
        direction = "A→B"
    elif -max_window <= delta < 0:
        direction = "B→A"
    else:
        direction = "none"
    
    return direction, abs(delta)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

def safe_normalize(series):
    """Normalize safely to [0,1], handling constants or NaNs."""
    if series.nunique() <= 1 or series.isna().all():
        return pd.Series(np.full(len(series), 0.5), index=series.index)
    scaler = MinMaxScaler()
    return pd.Series(scaler.fit_transform(series.values.reshape(-1, 1)).flatten(), index=series.index)


def postprocess_scores(df, alpha=0.7):
    """
    Normalize semantic + metadata scores AFTER building dataframe.
    Compute hybrid score globally across all candidates.
    """
    if df.empty:
        return df

    # Normalize globally (not per row)
    df["semantic_distance_norm"] = safe_normalize(df["semantic_distance"])
    df["metadata_score_norm"] = safe_normalize(df["metadata_score"])

    # Convert distance → similarity (invert scale)
    df["semantic_similarity"] = 1 - df["semantic_distance_norm"]

    # Weighted hybrid fusion
    df["hybrid_score"] = (
        alpha * df["semantic_similarity"] + (1 - alpha) * df["metadata_score_norm"]
    ).round(4)

    return df


##### Building Candidate Dependencies

Constructs dependency candidates using Chroma’s retriever and post-retrieval scoring. 

The function has the following parameters: 
df : the embeddings dataframe
collection: persistent chroma collection containing vectors
top_k: number of semantic neighbors to retrieve
meta_fields: metadata fields to use for correlation
alpha: weight for semantics vs metadata score fusion
max_time_window: temporal validation window in seconds

and returns the output:
candidates_df: candidate dependency 

In [ ]:
def infer_candidates(df, collection, top_k, meta_fields, alpha, max_time_window):

    meta_fields = meta_fields
    weights = compute_metadata_weights(df, meta_fields)

    candidates = []
    tqdm.write("🚀 Building hybrid dependency candidates via Chroma retriever...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating templates"):
        query_emb = np.array(row["embedding"], dtype=float)

        # NORMALIZING THE CASE (UPPER/LOWER) OF THE METADATA
        src_meta = normalize_metadata(row)
        results = semantic_retrieve_topk(collection, query_emb, top_k=top_k, metadata_filter=meta_fields   , include_distances=True)
        retrieved = results[0]

        # Loop through each retrieved match
        for j, doc_id in enumerate(retrieved["ids"]):
            # SKIP SELF-MATCHES
            if (
                doc_id == row["embedding_id"]
                or doc_id == row["template_id"]
                or row["template_id"] in row["embedding_id"]
                or row["embedding_id"].endswith(row["template_id"])
            ):
                continue

            target_meta = normalize_metadata(retrieved["metadatas"][j])
            semantic_dist = retrieved["distances"][j] if "distances" in retrieved else np.nan
            meta_score = metadata_correlation_score(src_meta, target_meta, meta_fields, weights)

            timestamp = safe_timestamp(target_meta)
            direction, delta = temporal_direction(row.get("Timestamp"), timestamp, max_time_window)
            relation_type = (
                "self-edge" if src_meta.get("component") == target_meta.get("component") else "inter_service"
            )

            # print("Source Component:", src_meta)
            # print("Target Componet:", target_meta)

            candidates.append({
                "src_id": row["template_id"],
                "target_id": doc_id,
                "src_component": src_meta.get("component"),
                "target_component": target_meta.get("component"),
                "relation_type": relation_type,
                "semantic_distance": float(semantic_dist),
                "metadata_score": meta_score,
                "temporal_relation": direction,
                "time_diff_seconds": delta,
            })
    
    candidates_df = pd.DataFrame(candidates)
    tqdm.write(f"✅ Built {len(candidates_df)} candidates.")
    return candidates_df


In [ ]:
candidates_df = infer_candidates(
    embeddings_df,
    collection=collection,
    top_k=5,
    meta_fields=["Pid", "Component", "Level"],
    alpha=0.7,   # weight for semantic importance
    max_time_window=5.0
)
candidates_df = postprocess_scores(candidates_df, alpha=0.7)
inferred_df = candidates_df[candidates_df["relation_type"] == "inter_service"]
# ✅ Inspect
print("🧩 Inter-Service Candidate Dependencies built:", len(inferred_df))
display(inferred_df.sort_values("hybrid_score", ascending=False).head(len(inferred_df)))
# print(inferred_df.info())
# print(inferred_df["semantic_distance"].describe())
# print(inferred_df["metadata_score"].describe())
# print(inferred_df["metadata_score"].describe())
# print(inferred_df["metadata_score"].value_counts().head(10))
# print(inferred_df["relation_type"].value_counts())
# print(inferred_df["relation_type"].unique())  



#### Inferred Candidate Dependency Graph

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

G = nx.DiGraph()

# Add edges with score attributes
for _, row in inferred_df.iterrows():
    G.add_edge(row["src_component"], row["target_component"], weight=row["hybrid_score"])

# Layout & edge color mapping
pos = nx.spring_layout(G, seed=42, k=0.7)
weights = np.array([d["weight"] for _, _, d in G.edges(data=True)])
norm = plt.Normalize(vmin=weights.min(), vmax=weights.max())
cmap = plt.cm.viridis

# Create figure and explicit axes so colorbar can be attached to the same axes
fig, ax = plt.subplots(figsize=(12, 8))

nodes = nx.draw_networkx_nodes(
    G, pos, ax=ax,
    node_size=800, node_color="lightgray", edgecolors="blue"
)
edges = nx.draw_networkx_edges(
    G, pos, ax=ax,
    edge_color=weights,
    edge_cmap=cmap,
    edge_vmin=weights.min(),
    edge_vmax=weights.max(),
    arrows=True,
    arrowsize=12
)
nx.draw_networkx_labels(G, pos, ax=ax, font_size=9, font_weight="bold", font_color="red")

# Add colorbar properly linked to the same axes
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(weights)  # associate actual values
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Hybrid Score")

ax.set_title("Inferred Inter-Service Dependency Graph")
ax.axis("off")
plt.tight_layout()
plt.show()